In [186]:
import pandas as pd
import numpy as np
import chess

import itertools

### Loading data and parsing the text file for moves, results and openings

In [198]:
game_list = [line for line in open('GMallboth.pgn')] 

all_moves_list = []
i = 11
while i <= (len(game_list)/20):           ###### NOTE: '/8' only using 1% of the data availible from grandmasters
    all_moves_list.append(game_list[i])
    i += 16
    
    
all_results = []
i = 13
while i <= (len(game_list)/20):
    all_results.append(game_list[i])
    i += 16
    
all_openings = []
i = 7
while i <= (len(game_list)/20):
    all_openings.append(game_list[i])
    i += 16

In [199]:
%%time
all_san_list = [get_move_list(game) for game in all_moves_list]
print(len(all_san_list))

13780
CPU times: user 638 ms, sys: 16.7 ms, total: 655 ms
Wall time: 717 ms


In [200]:
def white_results(result_string_list):
    '''
    Reformats and standardizes stripped result text from chess game in PGN/TXT format
    
    Input: list of strings - (results text with \n)
    Output: win, lose, draw in list format
    
    '''
    white_result_list = []
    for i in range(len(result_string_list)):
            result = result_string_list[i][:-1]
            result = result.replace('1-0', 'win')
            result = result.replace('0-1', 'lose')
            result = result.replace('1/2-1/2', 'draw')
            white_result_list.append(result)
    return(white_result_list)

In [201]:
%%time
white_test = white_results(all_results)

CPU times: user 15.1 ms, sys: 3.04 ms, total: 18.2 ms
Wall time: 22.2 ms


In [202]:
white_win = []
white_lose = []
white_draw = []

for i in range(len(white_test)):
    if white_test[i] == 'win':
        white_win.append(all_san_list[i])
    elif white_test[i] == 'lose':
        white_lose.append(all_san_list[i])
    else:
        white_draw.append(all_san_list[i])
print(len(white_win), len(white_lose), len(white_draw))

4118 2677 6985


In [203]:
### Maybe try (# apt install stockfish)
### stockfish 3.5.0 released Dec 14, 2019 ... maybe try a previous version....
### 

from stockfish import Stockfish

stockfish = Stockfish('Desktop/stockfish-11-mac/Mac/stockfish-11-64')

In [204]:
import chess
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")

#board = chess.Board()
#info = engine.analyse(board, chess.engine.Limit(time=0.1))
#print("Score:", info["score"])
# Score: +95

board = chess.Board()
print(board)
#info = engine.analyse(board, chess.engine.Limit(depth=10))
#print("Score:", info["score"])
# Score: #+1
#print(info['depth'], info['nodes'], info['time'])
engine.quit()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [205]:
def clean_fen(string):
    '''
    Takes a fen with extraneous features
    
    input: plain FEN notation i.e: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR
    output: 1x64 list - with piece abbriviation in position, open space as '1'
    '''
    string = string.replace('8','1'*8)
    string = string.replace('7','1'*7)
    string = string.replace('6','1'*6)
    string = string.replace('5','1'*5)
    string = string.replace('4','1'*4)
    string = string.replace('3','1'*3)
    string = string.replace('2','1'*2)
    string = string.replace('1','1'*1)
    string = string.replace('/','')
    string_list = [i for i in string]
    string_array = np.asarray(string_list)
    
    return(string_array)


def get_bitwise(board_state):
    '''
    Takes a [1x64] board state and creates 6 bitwise [1x64]
    '''
    bs = board_state #test ---- #First posiiton tuple  #winning_position[0][0] - for i in range:
    r_ray = np.zeros(64)
    n_ray = np.zeros(64)
    b_ray = np.zeros(64)
    q_ray = np.zeros(64)
    k_ray = np.zeros(64)
    p_ray = np.zeros(64)
    for i in range(64):
        if bs[i] == 'r':
            r_ray[i] = -1
        if bs[i] == 'R':
            r_ray[i] = 1 
        
        if bs[i] == 'b':
            b_ray[i] = -1
        if bs[i] == 'B':
            b_ray[i] = 1
        
        if bs[i] == 'n':
            n_ray[i] = -1
        if bs[i] == 'N':
            n_ray[i] = 1 
    
        if bs[i] == 'q':
            q_ray[i] = -1
        if bs[i] == 'Q':
            q_ray[i] = 1
    
        if bs[i] == 'k':
            k_ray[i] = -1
        if bs[i] == 'K':
            k_ray[i] = 1 
        
        if bs[i] == 'p':
            p_ray[i] = -1
        if bs[i] == 'P':
            p_ray[i] = 1 
    master_ray = np.array((r_ray.astype('int8'), n_ray.astype('int8'), b_ray.astype('int8'), q_ray.astype('int8'), k_ray.astype('int8'), p_ray.astype('int8')))
    return(master_ray)

In [206]:
test_games = all_san_list
import pickle

### Takes a list of games, with a list of move inside each game, and generates novel board states - StockFish chess engine will then evaluate the positional score of each state and pair it up with 

In [207]:

%%time
list_scores = []
engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")

for game in test_games:
    board = chess.Board()
    for i in game:
        try:
            info = engine.analyse(board, chess.engine.Limit(depth=8))
            x = str(info['score']).replace('#+', '1000')
            x = x.replace('#-', '-1000')
            list_scores.append((int(x), get_bitwise(clean_fen(board.board_fen())))) ### takes the
            board.push_san(i)
        except:
            continue
        
with open('bitwise_data.p', 'wb') as write_file:
    pickle.dump(list_scores, write_file)
### . chess.engine.PlayResult()

CPU times: user 1h 18min 57s, sys: 2min 28s, total: 1h 21min 25s
Wall time: 1h 50min 40s


In [208]:
len(list_scores)
import pickle
with open('bitwise_data.p', 'wb') as write_file:
    pickle.dump(list_scores, write_file)

In [ ]:
exernal_data= 

In [154]:
df = pd.DataFrame(list_scores, columns= ['cp_score', 'bitwise'])

In [155]:
# (r_ray.astype('int8'), n_ray.astype('int8'), b_ray.astype('int8'), q_ray.astype('int8'), k_ray.astype('int8'), p_ray.astype('int8')


In [156]:

for game in range(len(df)):
    for i in range(64):
        df[f'rook{i}'] = df['rook'][game][i]
        df[f'night{i}'] = df['night'][game][i]
        df[f'bishop{i}'] = df['bishop'][game][i]
        df[f'queen{i}'] = df['queen'][game][i]
        df[f'king{i}'] = df['king'][game][i]
        df[f'pawn{i}'] = df['pawn'][game][i]
df = df.drop(['bitwise', 'rook', 'night', 'bishop', 'queen', 'king', 'pawn'], axis = 1)

In [209]:
df.astype('int64')
df.head(50)

,cp_score,rook0,night0,bishop0,queen0,king0,pawn0,rook1,night1,bishop1,...,bishop62,queen62,king62,pawn62,rook63,night63,bishop63,queen63,king63,pawn63
0,48,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
1,20,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
2,135,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
3,17,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
4,50,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
5,0,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
6,65,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
7,5,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
8,23,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0
9,4,-1,0,0,0,0,0,0,-1,0,...,0,0,0,0,1,0,0,0,0,0


In [163]:
import xgboost
import sklearn.model_selection

import xgboost

from xgboost import plot_importance
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

In [212]:
X = df.drop('cp_score', axis=1)
y = df['cp_score']
print(len(X))
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=.9, random_state=2)

247345


In [168]:
XGBoost = XGBRegressor(max_depth=10, n_estimators=100)
XGBoost.fit(X_train, y_train)



[17:33:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [210]:
from sklearn import metrics
import statsmodels.api as sm
y_pred = XGBoost.predict(X_test)
#xgboost.loss_MSE_xgb(X_test)
#accuracy = accuracy_score(y_test, y_pred)
#XGBoost_score = round(accuracy, 4)
rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred))
#statsmodels.regression.linear_model.OLS(X_test, y_pred)


#model = statsmodels.OLS(y, X)
#results = model.fit()
rmse

398.1722319556766